In [1]:
import math as mt
import numpy as np
import pandas as pd

In [2]:
def invertArray(col_arr):
    res = [[] for i in range(len(col_arr[0]))]
    for c in range(len(col_arr)):
        for r in range(len(col_arr[0])):
            res[r].append(col_arr[c][r])
    return res



def getCommaIndex(inpt_str):
    commaInd_inpt = -1
    for i in range(len(inpt_str)):
        commaInd_inpt = i if inpt_str[i] == '.' else commaInd_inpt
    if commaInd_inpt ==-1:
        commaInd_inpt = len(inpt_str) - 1
    return commaInd_inpt

def sig_format(inpt, err, sig_dgts, smallDigits = True, debug = False):

    if debug:
        print("inpt = ", inpt)
    pot = 0
    if smallDigits and (np.abs(inpt) < 10**(-3) or np.abs(err) < 10**(-3)):
        pot = mt.ceil(np.max([0, -mt.log10(np.abs(inpt)) if inpt != 0 else 0, -mt.log10(np.abs(err)) if err != 0 else 0]))
        inpt *= 10**pot
        err *= 10**pot
    err_str = str(err)
    isSig = False
    lastSigInd = 0
    sigToGo = sig_dgts
    commaInd = -1
    for i in range(len(err_str)):
        if isSig == False and not err_str[i] in ['0', '.']:
            isSig = True
            if err_str[i] in ['1', '2'] and sig_dgts == 1:
                sigToGo += 1

        if err_str[i] == '.':
            commaInd = i
        else:
            sigToGo -= isSig
        if sigToGo > 0:
            lastSigInd += 1

    if sigToGo > 0:
        sig_dgts -= 1
        lastSigInd -= 1
    if commaInd == -1:
        commaInd = len(err_str)
    commaInd = commaInd if commaInd != -1 else len(err_str)
    dec_pot = - lastSigInd + commaInd - (lastSigInd < commaInd)
    err = err * 10**(-dec_pot)
    err = mt.ceil(err)
    if dec_pot <= 1:
        err_str = str(err * 10**(dec_pot))
    else:
        err_str = str(err) + "\cdot 10^{" + str(dec_pot) + "}"
    if lastSigInd > commaInd:
        err_str = err_str[:lastSigInd + 1]

    if debug:
        print("err =", err_str, "( as Number:", err, "*10^", dec_pot, ")")

    inpt *= 10**(-dec_pot)
    inpt_str = str(round(inpt) * 10**dec_pot)
    if debug:
        print("inpt_str = ", inpt_str, "\n", "inpt=", inpt)
    commaInd_inpt = getCommaIndex(inpt_str)

    if len(inpt_str) - 1 - commaInd_inpt != -dec_pot:
        inpt_str += '0'
    commaInd_inpt = getCommaIndex(inpt_str)


    inpt_str = inpt_str[:commaInd_inpt - dec_pot + 1]
    if commaInd_inpt - dec_pot + 1 < commaInd_inpt:
        inpt_str += " \cdot 10^{" + str(dec_pot - 1) +"}"
    if debug:
        print(inpt_str, "\pm", err_str)
    if pot != 0:
        inpt_str += ("\cdot 10^{" + str(-pot) + "}" if pot != 0 else "")
        err_str += ("\cdot 10^{" + str(-pot) + "}" if pot != 0 else "")
    return inpt_str, err_str


def sigFormatVal(val, err, sig_digits = 1, debug = False, output = True, outpAndReturn = False):

    if debug:
        print("val=", val, "err=", err)
    fmt = sig_format(val, err, sig_digits, debug = debug)

    res = "$ " + fmt[0] + " \pm " + fmt[1]
    if val != 0:
        if debug:
            print("val=", val, "err = ", err, "err/val = ", err/val)
            print(val/err * val)
        res += "\: (\pm " + sig_format(0, np.abs(100.0 * err/val), 2, smallDigits=False)[1] + "\%)"
    res += "$"
    if output:
        print(res)
        if not outpAndReturn:
            return

    return res



def isNumber(obj):
    try:
        tmp = int(obj)
        return True
    except:
        return False
#IMPORTANT NOTE: CANT HANDLE SIGNIFICANT 0 AS LAST DIGIT! (for example error 0.10 will be treated as 0.1)

def sigFormatTable(arr, val_err_indices, sig_digits = 1, int_cols = [], debug = False):
    res = [["undefined" for k in range(len(arr[i]))] for i in range(len(arr))]
    trashCan = []
    for i in range(len(arr)):
        for j in val_err_indices:
            if i == j[0]:
                for k in range(len(arr[i])):
                    if not isNumber(arr[i][k]):
                        res[i][k] = str(arr[i][k])
                        if debug:
                            print(res[i][k])
                        continue
                    fmt = sig_format(arr[i][k], arr[j[1]][k], sig_digits)
                    res[i][k] =  "$ " + fmt[0] + " \pm " + fmt[1]
                    if arr[i][k] != 0:
                        res[i][k] += "\: (\pm " + sig_format(0, np.abs(100.0 * arr[j[1]][k]/arr[i][k]), 2, smallDigits=False)[1] + "\%)"
                    res[i][k] += "$"
                trashCan.append(j[1])
                break
        if res[i][0] == "undefined":
            for k in range(len(arr[i])):
                if not isNumber(arr[i][k]):
                    res[i][k] = str(arr[i][k])
                    continue
                if i in int_cols:
                   res[i][k] = "$ " + str(int(arr[i][k])) + "$"
                else:
                    res[i][k] = "$ " + sig_format(arr[i][k], arr[i][k], sig_digits)[0] + "$"


    for e in trashCan:
        res.pop(e)
        for i in range(len(trashCan)):
            if trashCan[i] > e:
                trashCan[i] = trashCan[i] - 1
    return res

#IMPORTANT NOTE: CANT HANDLE SIGNIFICANT 0 AS LAST DIGIT! (for example error 0.10 will be treated as 0.1)

def asLatex(arr, col_labels, dontInvert = False):
    df = pd.DataFrame(invertArray(arr), columns=col_labels) if not dontInvert else pd.DataFrame(arr, columns = col_labels)
    cformate = "|"
    for i in df.columns:
        cformate += "l|"

    res = """\\begin{table}[]\n
    \t\centering\n
    \t\caption{CHANGE THIS LATER}\n\t""" + df.style.hide(axis="index").to_latex(column_format=cformate, hrules = True).replace("toprule", "hline").replace("\n", "\n\t\t") + "\t\label{tab:my_label}\n" + "\end{table}"
    res = res.replace("midrule", "hline")
    return res.replace("bottomrule", "hline")

def printTable(arr, col_labels):
    df = pd.DataFrame(invertArray(arr), columns=col_labels)
    print(df.to_markdown())

def showTable(arr, col_labels):
    df = pd.DataFrame(invertArray(arr), columns=col_labels)
    display(df)

In [3]:
T = [88, 32.88, 87.6]
sigT = 0.01

In [4]:
nr_b = 0
nr_c = 1
nr_d = 2

In [5]:
RC = [mt.log(9) * t for t in T]
sigRC = mt.log(9) * sigT


In [6]:
tbl = [["b", "c", "d"], [rc for rc in RC], [sigRC for rc in RC]]
R_2 = 2.01 *10**6
sig_R_2 = 0.01

print(asLatex(sigFormatTable(tbl, [[1, 2]]), ["Messung 3.", "RC in \Ohm \\un F"]))

\begin{table}[]

    	\centering

    	\caption{CHANGE THIS LATER}
	\begin{tabular}{|l|l|}
		\hline
		Messung 3. & RC in \Ohm \un F \\
		\hline
		b & $ 193.356 \pm 0.022\: (\pm 0.012\%)$ \\
		c & $ 72.245 \pm 0.022\: (\pm 0.031\%)$ \\
		d & $ 192.477 \pm 0.022\: (\pm 0.012\%)$ \\
		\hline
		\end{tabular}
			\label{tab:my_label}
\end{table}


In [7]:
#Calculate R_oszi:
R_oszi = R_2 * (RC[nr_b]/RC[nr_c] - 1)
sig_R_Oszi = (sig_R_2**2 * (RC[nr_b]/RC[nr_c] - 1)**2 + sigRC**2 * ((R_2/RC[nr_c])**2 + (R_2 * (RC[nr_b]/RC[nr_c]**2))**2))**0.5
sigFormatVal(R_oszi, sig_R_Oszi)




$ 336960 \cdot 10^{1} \pm 18\cdot 10^{2}\: (\pm 0.052\%)$


In [8]:
#Calcluate C:
R = [R_oszi, 1/(1/R_oszi + 1/R_2)]
sigR = [sig_R_Oszi, ((-1/(1/R_oszi + 1/R_2)**2)**2 * (sig_R_Oszi**2 * 1/R_oszi**4 + sig_R_2**2 * 1/R_2**4))**0.5]

C = [RC[i]/R[i] for i in range(len(R))][0]
sigC = [((sigRC**2)/R[i]**2 + sigR[i]**2 * RC[i]**2/R[i]**4)**0.5 for i in range(len(R))][0]
sigFormatVal(C, sigC)

$ 5738\cdot 10^{-8} \pm 4\cdot 10^{-8}\: (\pm 0.054\%)$


In [9]:
R_d = RC[nr_d]/C
sigR_d = (sigRC**2/C**2 + sigC**2 * RC[nr_d]**2/C**4)**0.5

R.append(R_d)
sigR.append(sigR_d)
R_x = 1/(1/R_d - 1/R_oszi)

sigR_x = ((R_oszi**2/(R_oszi - R_d)**2)**2 * sigR_d**2 + sig_R_Oszi**2 * (R_d**2 / (R_oszi - R_d)**2)**2)**0.5
    #((-1/(1/R_d - 1/R_oszi)**2)**2 * (sigR_d**2 * (-1/R_d**2)**2 + sig_R_Oszi**2 * (1/R_oszi**2)**2))**0.5

sigFormatVal(R_x, sigR_x)

$ 740 \cdot 10^{6} \pm 13\cdot 10^{7}\: (\pm 17\%)$


In [10]:

tbl = [["b", "c", "d"], [r for r in R], [sigr for sigr in sigR]]
print(asLatex(sigFormatTable(tbl, [[1, 2]]), ["Messung", "R in \Ohm"]))

\begin{table}[]

    	\centering

    	\caption{CHANGE THIS LATER}
	\begin{tabular}{|l|l|}
		\hline
		Messung & R in \Ohm \\
		\hline
		b & $ 336960 \cdot 10^{1} \pm 18\cdot 10^{2}\: (\pm 0.052\%)$ \\
		c & $ 1258990 \pm 250\: (\pm 0.02\%)$ \\
		d & $ 335420 \cdot 10^{1} \pm 19\cdot 10^{2}\: (\pm 0.055\%)$ \\
		\hline
		\end{tabular}
			\label{tab:my_label}
\end{table}
